In [1]:
# %uv pip install -U -q crewai crewai-tools

In [6]:
%uv pip install -U -q crewai[google-genai]

Note: you may need to restart the kernel to use updated packages.


In [1]:
from crewai import Agent, Task, LLM, Crew
from typing import Tuple, Union, Dict, Any
from crewai import TaskOutput
from datetime import date
from crewai_tools import SerperDevTool

In [2]:
import os

SERPER_API_KEY = os.getenv('SERPER_API_KEY')
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

if SERPER_API_KEY and GEMINI_API_KEY:
    os.environ['SERPER_API_KEY'] = SERPER_API_KEY
    os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY
    print('✅ API keys set successfully!')
else:
    raise ValueError('Please enter both SERPER and GEMINI API keys')

✅ API keys set successfully!


In [3]:
def validate_blog_content(result: TaskOutput) -> Tuple[bool, Any]:
    """Validate blog content meets requirements."""
    try:
        # Check word count
        try:
            word_count = len(result.raw.split())
            print(f"Word count: {word_count}")
            if word_count > 50:
                return (False, "Blog content exceeds 50 words")
        except Exception as wc_error:
            print(f"Error during word count check: {wc_error}")
            return (False, f"Error during word count check: {wc_error}")


        # Additional validation logic here
        return (True, result.raw.strip())
    except Exception as e:
        print(f"Unexpected error during validation: {e}")
        return (False, f"Unexpected error during validation: {e}")

In [4]:
# LLM to be used by the agent(s)
llm = LLM(model="gemini/gemini-3-flash-preview", api_key=GEMINI_API_KEY)

blog_agent = Agent(
    role="Blog Writer",
    goal="Write blog post",
    backstory="An expert blog writer",
    tools=[SerperDevTool()],
    llm=llm,
    verbose=True
)

blog_task = Task(
    description="Write a super DETAILED blog post about {prompt} for {year}",
    expected_output="""A properly structured blog post under 50 words.
    Blog format:
    # Title
    ## Subtitle
    Paragraphs...
    """,
    agent=blog_agent,
    markdown=True,
    guardrail=validate_blog_content,  # Add the guardrail function
    max_retries=4  # Set the maximum number of retries
)

crew = Crew(
    agents=[blog_agent],
    tasks=[blog_task],
    verbose=True
)

In [5]:
results = crew.kickoff(
    inputs={
        "prompt": "CrewAI",
        "year": date.today().year
    }
)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: ae2fc8d3-bccd-45d5-b0bf-fa6b96dae040                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Blog Writer                                                                                             │
│                                                                                                                 │
│  Task: Write a super DETAILED blog post about CrewAI for 2026                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

c:\Users\anoop\AppData\Local\Programs\Python\Python311\Lib\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')


c:\Users\anoop\AppData\Local\Programs\Python\Python311\Lib\site-packages\rich\live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Blog Writer                                                                                             │
│                                                                                                                 │
│  Thought: Thought: I need to research CrewAI's current trajectory and future predictions for 2026 to write an   │
│  insightful, albeit very brief, blog post.                                                                      │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🛡️ Guardrail Check ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Guardrail Evaluation Started                                                                                   │
│  Name: def validate_blog_content(result: TaskOutput) -> T...                                                    │
│  Status: 🔄 Evaluating                                                                                          │
│  Attempt: 1                                                                                                     │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── 🛡️ Guardrail Success ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Guardrail Passed                                                                                               │
│  Name: Validation Successful                                                                                    │
│  Status: ✅ Validated                                                                                           │
│  Attempts: 1                                                                                                    │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 9cad0126-d7f3-4e5f-9f72-8e5d15826fca                                                                     │
│  Agent: Blog Writer                                                                                             │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: ae2fc8d3-bccd-45d5-b0bf-fa6b96dae040                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: # CrewAI 2026: Agentic Mastery                                                                   │
│  ## Scaling Multi-Agent Systems                                                                                 │
│  In 2026, **CrewAI** becomes a decentralized operating system. Utilizing `autonomous-governance` and            │
│  `feedback-loops`, crews achieve 99% accuracy. Seamless integration with edge computing and private LLMs        │
│  ensures secure scalability. CrewAI is the definitive backbone for complex digital labor.                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯